In [1]:
%load_ext autoreload

%autoreload 2
import matplotlib.pyplot as plt
import pickle
import emcee
from astropy.coordinates import SkyCoord, match_coordinates_3d, match_coordinates_sky
import sys
import numpy as np
from astropy.table import Table, QTable, hstack, vstack
import iminuit
from iminuit import Minuit
import corner
import os
os.environ['CLMM_MODELING_BACKEND'] = 'nc'
import clmm
from clmm import Cosmology
from astropy.table import Table, QTable, hstack, vstack
import pyccl as ccl
from astropy.cosmology import FlatLambdaCDM
cosmo = Cosmology(H0 = 71.0, Omega_dm0 = 0.265 - 0.0448, Omega_b0 = 0.0448, Omega_k0 = 0.0)
cosmo_clmm = Cosmology(H0 = 71.0, Omega_dm0 = 0.265 - 0.0448, Omega_b0 = 0.0448, Omega_k0 = 0.0)
cosmo_ccl  = ccl.Cosmology(Omega_c=0.265-0.0448, Omega_b=0.0448, h=0.71, A_s=2.1e-9, n_s=0.96, Neff=0, Omega_g=0)
cosmo_astropy = FlatLambdaCDM(H0=71.0, Om0=0.265, Ob0 = 0.0448)

#module from my personnal github
sys.path.append('../modules/')
import analysis_Mass_Richness_relation as analysis
import CL_WL_miscentering as mis
import CL_WL_DATAOPS_make_profile as prf
import CL_WL_two_halo_term as twoh
import CL_WL_mass_conversion as utils
import CL_WL_fit_cluster_mass as fit
import CL_Likelihood_for_Mass_richness_relation as mr
import CL_fiducial_mass_richness_relation as fiducial
sys.path.append('/pbs/throng/lsst/users/cpayerne/ClusterLikelihoods/modules/')
import utils as ut

In [2]:
def load(filename, **kwargs):
    with open(filename, 'rb') as fin:
        return pickle.load(fin, **kwargs)

# Load data

In [3]:
profile_true = load('/pbs/throng/lsst/users/cpayerne/CLMassDC2/data/individual_profile_SkySim5000_gal_redMaPPer_cluster_epsilon_shape_true_z.pkl')

In [4]:
Z_bin = analysis.Z_bin
Obs_bin = analysis.Obs_bin

# Fit WL mass

## Estimating shear profiles & covariances

In [5]:
profile_true_stack = prf.stacked_profile(profile = profile_true,
                    r_in = 'radius_av',
                    gt_in = 'gt_av', gx_in = 'gx_av',
                    r_out = 'radius',
                    gt_out = 'gt', gx_out = 'gx',
                    weight = 'norm_sum',
                    z_name = 'cluster_z', obs_name = 'richness',
                    Z_bin = Z_bin, Obs_bin = Obs_bin)
covariance_true_stack = prf.bootstrap_covariance(profile = profile_true,
                    r_in = 'radius_av',
                    gt_in = 'gt_av', gx_in = 'gx_av',
                    r_out = 'radius',
                    gt_out = 'gt', gx_out = 'gx',
                    weight = 'norm_sum',
                    n_boot = 100,
                    z_name = 'cluster_z', obs_name = 'richness',
                    Z_bin = Z_bin, Obs_bin = Obs_bin)

# Fit mass-richness relation
## Fit mass then fit mass-richness relation
### Fit mass

In [6]:
down = 1
up = 5.5
a = 0
b = 1
true = fit.fit_WL_cluster_mass(profile = profile_true_stack, covariance = covariance_true_stack, a = a, b = b, rmax = up, two_halo_term = False, fix_c = True)

fitting...


### Fit mass-richness relation

In [7]:
def lnL(theta, m200c_mean, m200c_err_mean, richness_mean, z_mean):
    return mr.lnL_WL_binned(theta, m200c_mean, m200c_err_mean, richness_mean, z_mean, z0, richness0)

In [8]:
fits=[true]
z0 = analysis.z0
richness0 = analysis.richness0
initial_binned = [14.1,-0,.75]
npath = 100
ndim=3
nwalkers = 800
pos_binned = initial_binned + 0.01 * np.random.randn(nwalkers, len(initial_binned))
label = ['true']
sampler_list = {name : 0 for name in label}
for i,fit_true in enumerate(fits):
    m200c_mean = 10**np.array(fit_true['logm200_w'])
    m200c_err_mean =np.array(fit_true['logm200_w_err'])*np.log(10)*m200c_mean
    richness_mean = np.array(fit_true['obs_mean'])
    z_mean = np.array(fit_true['z_mean'])
    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnL, args = (m200c_mean, m200c_err_mean, richness_mean, z_mean))
    sampler.run_mcmc(pos_binned, npath,progress=True)
    sampler_wl = sampler.get_chain(discard = 0, flat = True)
    sampler_list[label[i]] = sampler_wl

emcee: Exception while calling your likelihood function:
  params: [ 1.40733982e+01 -7.88454496e-03  7.35185619e-01]
  args: (array([1.27413671e+14, 1.82665437e+14, 2.93918962e+14, 5.25295662e+14,
       1.18279396e+14, 2.03394667e+14, 2.97196170e+14, 3.87563216e+14,
       1.25201459e+14, 1.71715113e+14, 2.94798129e+14, 3.63165698e+14,
       1.34717668e+14, 1.70239039e+14, 2.64312582e+14, 4.22489949e+14,
       1.21422524e+14, 1.37408661e+14, 2.17588149e+14, 4.75967208e+14]), array([4.02228153e+12, 5.74528981e+12, 1.08203383e+13, 3.60014865e+13,
       3.25076860e+12, 6.26886410e+12, 1.10946611e+13, 2.49169373e+13,
       3.68407292e+12, 6.81536347e+12, 1.20892274e+13, 2.40296215e+13,
       4.73899881e+12, 7.72582763e+12, 1.47389807e+13, 3.93914276e+13,
       6.27324953e+12, 1.02057611e+13, 2.03873567e+13, 4.12767948e+13]), array([ 25.407982,  40.64189 ,  69.044014, 156.64774 ,  25.32449 ,
        41.3784  ,  64.492226, 134.93892 ,  25.338959,  40.64467 ,
        68.394394, 125.923

Traceback (most recent call last):
  File "/pbs/home/c/cpayerne/.conda/envs/mydesc/lib/python3.7/site-packages/emcee/ensemble.py", line 545, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "<ipython-input-7-605f7f8063c5>", line 2, in lnL
    return mr.lnL_WL_binned(theta, m200c_mean, m200c_err_mean, richness_mean, z_mean, z0, richness0)
AttributeError: module 'CL_Likelihood_for_Mass_richness_relation' has no attribute 'lnL_WL_binned'


AttributeError: module 'CL_Likelihood_for_Mass_richness_relation' has no attribute 'lnL_WL_binned'

## Fit mass-richness relation directly

In [ ]:
moo=clmm.Modeling(massdef = 'critical', delta_mdef = 200, halo_profile_model = 'nfw')
moo.set_cosmo(cosmo_clmm)

In [ ]:
def logmh(logrichness, z, logM0, alpha, beta):
    return mr.mu_logM_lambda_f(z, logrichness, logM0, alpha, beta, z0, richness0)

In [ ]:
def ds(r, m, c, z):
    moo.set_mass(m), moo.set_concentration(c)
    y=moo.eval_excess_surface_density([float(r[i]) for i in range(len(r))], z)
    return y

In [ ]:
def lnlikelihood(theta):
    logM0,G,F=theta
    chi2=0
    for n, halo in enumerate(profile_true_stack):
        m = 10**logmh(np.log10(halo['obs_mean']), halo['z_mean'], logM0, G, F)
        mask = (halo['radius'] > 1)*(halo['radius'] < 5)
        y=ds(halo['radius'], m, 4 , halo['z_mean'])
        chi2=chi2+.5*np.sum(((y-halo['gt'])**2/covariance_true_stack[n]['cov_t'].diagonal())[mask])
    return -chi2

In [ ]:
print(lnlikelihood([14,0,1]))

In [ ]:
from multiprocessing import Pool
nwalker=100
pos=np.random.randn(nwalker,3) + np.array([14,0,1])
with Pool() as pool:
    sampler = emcee.EnsembleSampler(nwalker, 3, lnlikelihood,pool=pool)
    sampler.run_mcmc(pos, 100, progress=True);

In [ ]:
flat_samples = sampler.get_chain(discard=50, thin=3, flat=True)
plt.rcParams['axes.linewidth'] = 1.3
sigma1 = 1. - np.exp(-(1./1.)**2/2.)
sigma2 = 1. - np.exp(-(2./1.)**2/2.)
sigma3 = 1. - np.exp(-(3./1.)**2/2.)
truth = [14.162, -0.23, 0.8]
bins = 25
labels = [r'$\log_{10}(M_{0})$', r'$G_z$',r'$F_\lambda$']
c = ['salmon','m','k','orange','cyan']
fill_contours = [False, False, False,False,  True]
fig, axs = plt.subplots(len(labels), len(labels), figsize = (7,7))

fig = corner.corner(
flat_samples,
bins=bins, levels=(sigma1, sigma2),
fig = fig,
color = 'r',
smooth1d=True,smooth=True,plot_datapoints=False,
fill_contours=True, labels = labels,            
label_kwargs={"fontsize": 20},
use_math_text=True,
show_title = True,
plot_density=False,
max_n_ticks = 5);

fig = corner.corner(
sampler_list['true'][50000:a-1],
bins=bins, levels=(sigma1, sigma2),
fig = fig,
color = 'k',
smooth1d=True,smooth=True,plot_datapoints=False,
fill_contours=True, labels = labels,            
label_kwargs={"fontsize": 20},
use_math_text=True,
show_title = True,
plot_density=False,
max_n_ticks = 5);
    
for i in range(len(labels)):
    for j in range(len(labels)):
        axs[i,j].tick_params(axis='both', which = 'major', labelsize= 13)
        axs[i,j].legend(frameon = False, fontsize = 15)

In [ ]:
mean = np.mean(flat_samples, axis=0)

In [ ]:
mean

In [ ]:
for n, halo in enumerate(profile_true_stack):
    mh_=10**mh(np.log10(halo['obs_mean']), halo['z_mean'],*mean)
    plt.plot(halo['radius'], halo['gt'])
    plt.plot(halo['radius'],ds(halo['radius'], mh_, 4, halo['z_mean']))
plt.loglog()